In [ ]:
import panda as pd

# Klasifikácia zvieracích zvukov 

__ČASŤ MODEL__

__Autori__: Simona Glombíková, Dominik Jurečko

## Získanie dát
Dáta sú získavané zo súboru AnimalSound.csv. nachádzajú sa tu dvas stĺpce - *classname* a *src*.
Po načítaní dát je dôležité ich ešte upraviť na spracovateľný formát. 

In [ ]:
data = pd.read_csv('src/data/AnimalSound.csv')
print(data)

 ## Formát dát
 
 Formát dát potrebný na trénovanie je ukázaný v časti dataset. Ako vstup do modelu
 prichádzajú dvojrozmerné polia, ktoré predstavujú vyčíslené mfcc spektogramy.
 Takto spracované daáta eŠte rozdelíme na trénovací a testovací dataset. 
 

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs --bind_all

## Model

Navrhnutý model má takúto štruktúru, už po prvých pokusoch trénovania sme zistili, že bude potrebná
iterácia a zlepšenie modelu. Tento model sme vybrali z podobných prác, ktoré sme predstavovali v prvom odovzdaní. 

In [ ]:
class SoundAnimalDetector(keras.Model):
    def __init__(self, dim_output):
        super(SoundAnimalDetector, self).__init__()

        self.model_layers = [
            keras.layers.Conv2D(
                filters=1,
                kernel_size=3,
                padding='same',
                activation='relu'),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),

            keras.layers.Conv2D(
                filters=32,
                kernel_size=3,
                padding='same',
                activation='relu'),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),

            keras.layers.Conv2D(
                filters=64,
                kernel_size=3,
                padding='same',
                activation='relu'),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),

            keras.layers.Flatten(),

            keras.layers.Dense(
                units=512,
                activation='relu'),

            keras.layers.Dense(
                units=dim_output,
                activation='softmax')
        ]

    def call(self, x):
        for layer in self.model_layers:
            x = layer(x)
            return x


## Trénovanie modelu

Trénovanie modelu na klasifikáciu zvierat do druhov je zobrazené na kóde nižšie.

In [ ]:
import tensorflow.keras as keras
import os
import datetime

model = SoundAnimalDetector(dim_output=len(x_train))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train,
          y_train,
          batch_size=num_batch_size,
          epochs=num_epochs,
          validation_data=(x_test, y_test),
          callbacks=[keras.callbacks.TensorBoard(
                  log_dir=os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S")),
                  histogram_freq=1,
                  profile_batch=0
          )],
           verbose=1)
              
model.summary()